# More NumPy

In [ ]:
import numpy as np

## Copying Arrays

simply using "=" does not make a copy, but much like with lists, you will just have multiple names pointing to the same ndarray object

Therefore, we need to understand if two arrays, `A` and `B` point to:
* the same array, including shape and data/memory space
* the same data/memory space, but perhaps different shapes (a _view_)
* a separate cpy of the data (i.e. stored completely separately in memory)

All of these are possible:
* `B = A`
  
  this is _assignment_.  No copy is made. `A` and `B` point to the same data in memory and share the same shape, etc.  They are just two different labels for the same object in memory
  

* `B = A[:]`

  this is a _view_ or _shallow copy_.  The shape info for A and B are stored independently, but both point to the same memory location for data
  
  
* `B = A.copy()`

  this is a _deep_ copy.  A completely separate object will be created in memory, with a completely separate location in memory.
  
Let's look at examples

In [ ]:
a = np.arange(10)
print(a)


Here is assignment&mdash;we can just use the `is` operator to test for equality

In [ ]:
b = a
b is a

Since `b` and `a` are the same, changes to the shape of one are reflected in the other&mdash;no copy is made.

In [ ]:
b.shape = (2, 5)
print(b)
a.shape

In [ ]:
b is a

In [ ]:
print(a)

a shallow copy creates a new *view* into the array&mdash;the _data_ is the same, but the array properties can be different

In [ ]:
a = np.arange(12)
c = a[:]
a.shape = (3,4)

print(a)
print(c)

since the underlying data is the same memory, changing an element of one is reflected in the other

In [ ]:
c[1] = -1
print(a)

Even slices into an array are just views, still pointing to the same memory

In [ ]:
d = c[3:8]
print(d)

In [ ]:
d[:] = 0 

In [ ]:
print(a)
print(c)
print(d)

There are lots of ways to inquire if two arrays are the same, views, own their own data, etc

In [ ]:
print(c is a)
print(c.base is a)
print(c.flags.owndata)
print(a.flags.owndata)

to make a copy of the data of the array that you can deal with independently of the original, you need a _deep copy_

In [ ]:
d = a.copy()
d[:,:] = 0.0

print(a)
print(d)

## Boolean Indexing

There are lots of fun ways to index arrays to access only those elements that meet a certain condition

In [ ]:
a = np.arange(12).reshape(3,4)
a

Here we set all the elements in the array that are > 4 to zero

In [ ]:
a[a > 4] = 0
a

and now, all the zeros to -1

In [ ]:
a[a == 0] = -1
a

In [ ]:
a == -1

if we have 2 tests, we need to use `logical_and()` or `logical_or()`

In [ ]:
a = np.arange(12).reshape(3,4)
a[np.logical_and(a > 3, a <= 9)] = 0.0
a

Our test that we index the array with returns a boolean array of the same shape:

In [ ]:
a > 4

<div class="alert alert-block alert-info"><h3><span class="fa fa-flash"></span> Quick Exercise:</h3>

Create an array with 10 rows and 7 columns, and initialize it with random numbers (take a look at `np.random`)
    
Now compute the average of the array and replace all elements that are larger than the average with `0`.
    
</div>

## Avoiding Loops

In general, you want to avoid loops over elements on an array.

Here, let's create 1-d x and y coordinates and then try to fill some larger array

In [ ]:
M = 32
N = 64
xmin = ymin = 0.0
xmax = ymax = 1.0

x = np.linspace(xmin, xmax, M, endpoint=False)
y = np.linspace(ymin, ymax, N, endpoint=False)

print(x.shape)
print(y.shape)

we'll time out code

In [ ]:
import time

In [ ]:
t0 = time.time()

g = np.zeros((M, N))

for i in range(M):
    for j in range(N):
        g[i,j] = np.sin(2.0*np.pi*x[i]*y[j])
        
t1 = time.time()
print("time elapsed: {} s".format(t1-t0))

Now let's instead do this using all array syntax.  

First will extend our 1-d coordinate arrays to be 2-d.  NumPy has a function for this (`meshgrid()`)

In [ ]:
x2d, y2d = np.meshgrid(x, y, indexing="ij")

print(x2d[:,0])
print(x2d[0,:])

print(y2d[:,0])
print(y2d[0,:])

Now we'll retime this -- note: I'm including the meshgrid call in the timing to be fair

In [ ]:
t0 = time.time()
x2d, y2d = np.meshgrid(x, y, indexing="ij")
g2 = np.sin(2.0*np.pi*x2d*y2d)
t1 = time.time()
print("time elapsed: {} s".format(t1-t0))

In [ ]:
print(np.max(np.abs(g2-g)))

### Numerical differencing on NumPy arrays

Now we want to construct a derivative, 
$$
\frac{d f}{dx}
$$

In [ ]:
x = np.linspace(0, 2*np.pi, 25)
f = np.sin(x)

We want to do this without loops&mdash;we'll use views into arrays offset from one another.  Recall from calculus that a derivative is approximately:
$$
\frac{df}{dx} = \frac{f(x+h) - f(x)}{h}
$$
Here, we'll take $h$ to be a single adjacent element

In [ ]:
dx = x[1] - x[0]
dfdx = (f[1:] - f[:-1])/dx

In [ ]:
dfdx